In [ ]:
import util

train_ds,val_ds = util.get_train_val_set()

In [ ]:
for data in train_ds.take(1):
    print(data['image'].shape)

In [ ]:
def preprocess_data(data):
    image = tf.cast(data['image'],tf.float32)/255.0 
    label = tf.one_hot(data['label'],100)
    return image,label

In [ ]:
def augment_data(images,labels):
    images = tf.image.random_flip_left_right(images)
    images = tf.image.random_flip_up_down(images)
    images = tf.image.rot90(images)
    return (images, labels)

In [ ]:
train = train_ds.map(preprocess_data).batch(32).map(augment_data)
val = val_ds.map(preprocess_data).batch(32)

In [ ]:
%autoreload 2
import model 
exp_model = model.vanilla_model()

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=False)
early_stopping = tf.keras.callbacks.EarlyStopping(patience=5, monitor="val_loss",restore_best_weights=True,start_from_epoch=10)

exp_model.compile(optimizer=optimizer,loss=loss_fn)

In [ ]:
history = exp_model.fit(train, epochs=1, validation_data=val, callbacks=[early_stopping])

In [ ]:
plt.plot(history.history['loss'],label="training_loss")
plt.show()

In [ ]:
predictions = exp_model.predict(val)



In [ ]:
predictions = predictions.argmax(axis=-1)

In [ ]:
truth = tf.argmax(tf.concat([labels for _,labels in val],axis=0),axis=-1)
truth.shape


In [ ]:
from sklearn.metrics import accuracy_score 

accuracy_score(truth,predictions)

In [ ]:
import os
os.makedirs("./results",exist_ok=True)

In [ ]:
import datetime

In [ ]:
datetime.datetime.now().strftime("%Y%m%d%H%M%S")

In [ ]:
import json
json.dump([10,20],open("./hyp.json","w"))

In [ ]:
exp_model.save("best.keras")


In [ ]:

ex = tf.keras.models.load_model("./best.keras")

In [ ]:
import util
util.evaluate(ex,val)

In [ ]:
%run main.py


In [ ]:
test_model = tf.keras.models.load_model("./results/20241017175007/best.keras")

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import util

util.evaluate_on_test(test_model,top_k=5)

In [ ]:
test_ds = test_set.map(util.preprocess_data).batch(1)

In [ ]:
predictions = test_model.predict(test_ds)


In [ ]:
truth = tf.concat([tf.math.argmax(y,axis=-1) for _, y in test_ds], axis=0)

In [ ]:
truth.shape

In [ ]:
from sklearn.metrics import top_k_accuracy_score

top_k_accuracy_score(truth, predictions,k=5)

In [ ]:
tf.__version__

In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
%run main.py

2024-10-17 19:36:24.123551: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-17 19:36:25.386202: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30972 MB memory:  -> device: 0, name: Tesla V100-PCIE-32GB, pci bus id: 0000:5e:00.0, compute capability: 7.0
2024-10-17 19:36:26.645648: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/100


2024-10-17 19:36:28.133488: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8902
2024-10-17 19:36:28.423116: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-10-17 19:36:28.423799: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-10-17 19:36:28.423814: W tensorflow/stream_executor/gpu/asm_compiler.cc:77] Couldn't get ptxas version string: Internal: Couldn't invoke ptxas --version
2024-10-17 19:36:28.424237: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-10-17 19:36:28.424287: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Internal: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


1250/1250 [==============================] - 9s 4ms/step - loss: 4.1976 - val_loss: 4.1633
Epoch 2/100
1250/1250 [==============================] - 5s 4ms/step - loss: 3.7379 - val_loss: 4.1230
Epoch 3/100
1250/1250 [==============================] - 5s 4ms/step - loss: 3.4891 - val_loss: 4.1349
Epoch 4/100
1250/1250 [==============================] - 5s 4ms/step - loss: 3.3396 - val_loss: 4.1195
Epoch 5/100
1250/1250 [==============================] - 5s 4ms/step - loss: 3.2248 - val_loss: 4.0959
Epoch 6/100
1250/1250 [==============================] - 5s 4ms/step - loss: 3.1293 - val_loss: 4.0567
Epoch 7/100
1250/1250 [==============================] - 5s 4ms/step - loss: 3.0408 - val_loss: 4.0379
Epoch 8/100
1250/1250 [==============================] - 5s 4ms/step - loss: 2.9571 - val_loss: 3.9774
Epoch 9/100
1250/1250 [==============================] - 5s 4ms/step - loss: 2.8840 - val_loss: 3.9414
Epoch 10/100
1250/1250 [==============================] - 5s 4ms/step - loss: 2.8204 

In [2]:
best_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 256)       7168      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 256)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 128)       295040    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 8, 128)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 64)          73792     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 8, 32)          18464     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 4, 32)          0